In [111]:
import geopandas as gpd
from pathlib import Path
import numpy as np
import pandas as pd

import pickle
import src.modeling_turns as modeling_turns
import src.add_attributes as add_attributes
import src.prepare_network as prepare_network

import sys
sys.path.insert(0,str(Path.cwd().parent))
import file_structure_setup
config = file_structure_setup.filepaths()

In [112]:
unfiltered_turns = pd.read_parquet(config['network_fp']/'turns_df.parquet')
unfiltered_directed_links = pd.read_parquet(config['network_fp']/'directed_edges.parquet')
unfiltered_links = gpd.read_file(config['network_fp']/'final_network.gpkg',layer='edges')
unfiltered_nodes = gpd.read_file(config['network_fp']/'final_network.gpkg',layer='nodes')

In [113]:
unfiltered_links['oneway'].isna().any()

False

In [114]:
# Remove wrongway from directed links and turns
oneway_dict = dict(zip(links['linkid'],links['oneway']))
unfiltered_directed_links['wrongway'] = (unfiltered_directed_links['linkid'].map(oneway_dict) == True) & (directed_links['reverse_link']==True)
directed_links = unfiltered_directed_links[unfiltered_directed_links['wrongway']==False]
unfiltered_turns['source_oneway'] = unfiltered_turns['source_linkid'].map(oneway_dict)
unfiltered_turns['target_oneway'] = unfiltered_turns['target_linkid'].map(oneway_dict)
source_wrongway = ((unfiltered_turns['source_oneway'] == True) & (unfiltered_turns['source_reverse_link'] == True)) == False
target_wrongway = ((unfiltered_turns['target_oneway'] == True) & (unfiltered_turns['target_reverse_link'] == True)) == False
unfiltered_turns = unfiltered_turns[source_wrongway & target_wrongway]

In [115]:
print(unfiltered_links.shape[0],'links')
node_count = unfiltered_links['A'].append(unfiltered_links['B']).nunique()
print(unfiltered_directed_links.shape[0],'directed links')
print(node_count,'nodes')
print(unfiltered_turns.shape[0],'turns')

153157 links
306314 directed links
123628 nodes
514193 turns


# Filtered Network

In [116]:
#export calibration network
with (config['calibration_fp']/"calibration_network.pkl").open('rb') as fh:
    links, turns = pickle.load(fh)
undirected_links = links[links['reverse_link']==False]

In [117]:
print(links['linkid'].nunique(),'links')
node_count = links['A'].append(links['B']).nunique()
print(links.shape[0],'directed links')
print(node_count,'nodes')
print(turns.shape[0],'turns')

76912 links
144458 directed links
66151 nodes
222764 turns


Undirected Attributes

In [118]:
undirected_links.to_file(config['network_fp']/'network_stats.gpkg',layer='undirected_attributes')

In [133]:
undirected_links['length_mi']

0         0.03
3         0.21
4         0.07
6         0.02
8         0.07
          ... 
153814    0.09
153817    0.06
153819    0.22
153820    0.01
153823    0.07
Name: length_mi, Length: 76912, dtype: float64

In [148]:
miles = undirected_links.groupby('lanes')['length_mi'].sum().round(0)
counts = undirected_links['lanes'].value_counts()
pct = (miles / undirected_links['length_mi'].sum() * 100).round(0)
pct.name = 'pct'
pd.concat([counts,miles,pct],axis=1)

,lanes,length_mi,pct
0.0,11775,332.0,10.0
1.0,52570,2700.0,79.0
2.0,8445,257.0,8.0
3.0,4122,112.0,3.0


In [150]:
undirected_links.loc[undirected_links['speed']==0,'speed'] = 1

In [151]:
miles = undirected_links.groupby('speed')['length_mi'].sum().round(0)
counts = undirected_links['speed'].value_counts()
pct = (miles / undirected_links['length_mi'].sum() * 100).round(0)
pct.name = 'pct'
pd.concat([counts,miles,pct],axis=1)

,speed,length_mi,pct
1.0,18632,588.0,17.0
2.0,40680,2176.0,64.0
3.0,14902,525.0,15.0
4.0,2698,112.0,3.0


In [120]:
undirected_links['speed'].value_counts()

2.0    40680
3.0    14902
0.0    11775
1.0     6857
4.0     2698
Name: speed, dtype: int64

In [121]:
undirected_links['AADT'].describe()

count     76912.000000
mean       5214.421521
std        8508.487765
min           0.000000
25%         406.000000
50%        2107.000000
75%        4690.000000
max      131000.000000
Name: AADT, dtype: float64

In [122]:
undirected_links['truck_pct'].describe()

count    76912.000000
mean         0.175343
std          0.876012
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         28.000000
Name: truck_pct, dtype: float64

Directed

In [123]:
min_grade = links.groupby('linkid')['ascent_grade_%'].idxmax()
links.loc[min_grade,['linkid','name','highway','ascent_grade_%','geometry']].to_file(config['network_fp']/'network_stats.gpkg',layer='elevation')

In [124]:
ranking = {
    'multi use path':0,
    'bike lane':3,
    'buffered bike lane':2,
    'cycletrack':1
}
bicycle_facilities = links.loc[links['facility_fwd'].notna(),['linkid','name','highway','facility_fwd','geometry']].copy()
bicycle_facilities = bicycle_facilities[bicycle_facilities.drop(columns=['geometry']).duplicated()==False]
bicycle_facilities['facility_ranking'] = bicycle_facilities['facility_fwd'].map(ranking)
bicycle_facilities_idx = bicycle_facilities.groupby('linkid')['facility_ranking'].idxmin()
bicycle_facilities = bicycle_facilities.loc[bicycle_facilities_idx]
bicycle_facilities.to_file(config['network_fp']/'network_stats.gpkg',layer='bicycle_facilities')